In [2]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import happybase
import warnings
import sklearn.model_selection
import sklearn.preprocessing
import csv
import json
import pickle

In [2]:
data_dir =  r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [3]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [4]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 55.1 s, sys: 2.47 s, total: 57.6 s
Wall time: 57.6 s


In [2]:
#
with open("multi_hot_column_odict.pkl","rb") as f:
    multi_hot_column_odict = pickle.load(f)
with open("y_onehot.pkl","rb") as f:
    y_onehot = pickle.load(f)
#with tf.device('/device:GPU:1'):
#    model = tf.keras.models.load_model("model_output")

In [3]:
model = tf.keras.models.load_model("model_output")

In [5]:
y_onehot.categories_

[array(['1', '2'], dtype=object)]

In [6]:
multi_hot_column_odict.keys()

odict_keys(['关注的马甲', '关注的作者'])

In [11]:
y_onehot.classes_

array(['21~25', '26~29', '30~35', '36~45', '46~50', '50~', '~20'],
      dtype='<U5')

In [7]:
predict_df = df[list(multi_hot_column_odict.keys()) + ["用户id"]]
del df

In [8]:
predict_df

,关注的马甲,关注的作者,用户id
0,-1,-1,00001D26FE991A4BC4A01325D5E222DE
1,-1,-1,00006C3EF7F79B0F37EEFB8381505530
2,-1,-1,0000D0E1960239E8362F6B4100D9790A
3,-1,-1,0000f55e60d5ba7b36004429b235c95096f7153d
4,-1,-1,00014E9D5CF948B3FE6376564B23F64F
...,...,...,...
15703260,-1,-1,qlu2xsthqk7tp0mncpgr
15703261,-1,-1,qlu30bysbp1mj150uc1m
15703262,-1,-1,qlu30t7n6f2b1jgzexv
15703263,-1,-1,qlu315b37v5rc273amb0


In [ ]:
%%time
#测试数据
warnings.filterwarnings('ignore')
output_csv = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/age.csv"
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['age', 'udid' , "probility"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i,chunk in enumerate(np.array_split(predict_df, 30000)):
        udid_list = chunk["用户id"].values.tolist()
        predict_x = []
        for index in chunk.index:
            temp_x_list = []
            for j,column in enumerate(multi_hot_column_odict):
                split_column_list = chunk.loc[index,column].split(",") if isinstance(chunk.loc[index,column],str) else [str(predict_df.loc[index,column])]
                temp_x_list.extend(multi_hot_column_odict[column].transform([split_column_list]).tolist()[0])
            predict_x.append(temp_x_list)
        y_predict = model.predict([predict_x])
        #print(y_predict)
        for y_predict_i,udid in zip(y_predict[0],udid_list):
            writer.writerow({'age':y_onehot.inverse_transform(np.array([y_predict_i]))[0], 'udid': udid , 'probility':json.dumps(y_predict_i.tolist())})
        if i % 300 == 0:
            print(i)
        #break

0


In [ ]:
import happybase
output_df = pd.read_csv(output_csv)

In [ ]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(output_df.index):
            batch.put(output_df.loc[index,"udid"].encode(), {"attribute:predict_gender".encode(): output_df.loc[index,"gender"].encode()})
            #break
            if i % 100000 == 0:
                print(i)